# Шаблон решения
* Простой способ разработать своё первое решение и сгенерировать файл на проверку
* Дублирует функции из baseline
* Следовать ему не обязательно

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

# Прочитаем данные 

In [ ]:
data = pd.DataFrame.from_csv("./train.csv")

In [ ]:
x_col = [ 'dist','due', 'lat','lon','f_class','s_class','t_class',]
X_raw = data[x_col]
y = data['burned'].values
w = data['weights'].values

## Предобработка данных

In [ ]:
from sklearn.feature_extraction import DictVectorizer

#вытаскиватель категориальных фичей
vectorizer = DictVectorizer(sparse=False,dtype=np.bool)

def preprocess_data(X_raw):
    
    <Как нибудь предобработать данные>
    
    return <предобработанные данные>

In [ ]:
X = preprocess_data(X_raw)

## Поделим на обучение и валидацию
* Важно, что мы делим не случайно, а по времени, т.к. тестовые данные по времени дальше обучающих

In [ ]:
# Как-нибудь порезать данные
Xtr = X[:1000000]
Xval = X[1000000:]
Ytr = y[:1000000]
Yval = y[1000000:]
Wtr = w[:1000000]
Wval = w[1000000:]


## Обучим модель

In [ ]:


model = <Как-нибудь обученная модель>
model.fit(Xtr,Ytr,sample_weight=Wtr)


## Оценим качество

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import roc_curve


for source_i, Xi,Yi,Wi in [
                            ["train",Xtr,Ytr,Wtr],
                            ["val",Xval,Yval,Wval]
                                ]:
    
    # Предскажем вероятность сгорания
    Yi_pred_proba = model.predict_proba(Xi)[:,1]
    
    #Поделим предсказание на сгоревшие и не сгоревшие по порогу (0.5)
    thr = 0.5
    Yi_pred_class = Yi_pred_proba>thr

    auc = roc_auc_score(Yi,Yi_pred_proba,sample_weight = Wi)
    acc = accuracy_score(Yi,Yi_pred_class,sample_weight = Wi)
    prc = precision_score(Yi,Yi_pred_class,sample_weight = Wi)
    rcl = recall_score(Yi,Yi_pred_class,sample_weight = Wi)
    
    print '%s: \t AUC = %.5f \t Accuracy = %.5f \t Precision = %.5f \t Recall = %.5f'%(source_i, auc, acc,prc,rcl)
    
    fpr,tpr,_ = roc_curve(Yi,Yi_pred_proba,sample_weight=Wi)
    
    plt.plot(fpr,tpr,label = source_i)
    
plt.legend()


In [ ]:
def plot_forest_feature_importances(rf,feature_names):
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. %s (%f)" % (f + 1, feature_names[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    
    plt.yscale("log")
    plt.show()

In [ ]:
plot_forest_feature_importances(model,X.columns)

## Отгрузим решение в файл

In [ ]:
def generate_submission(preproc_fun,model,filename="submission.csv",threshold = 0.5):
    
    data_eval = pd.DataFrame.from_csv("./evaluation.handout.csv")
    
    
    x_col = [ 'dist','due', 'lat','lon','f_class','s_class','t_class',]
    X_eval = preproc_fun(data_eval[x_col])
    
    # Предскажем вероятность сгорания
    Y_pred_proba_eval = model.predict_proba(X_eval)[:,1]
    
    #Поделим предсказание на сгоревшие и не сгоревшие по порогу (thr)
    
    Y_pred_class_eval = Y_pred_proba_eval>threshold
    
    
    response = pd.DataFrame()
    response["Y_class"] = Y_pred_class_eval.astype(int)
    response["Y_prob"] = map("{0:.5f}".format,Y_pred_proba_eval)
    
    
    response.to_csv(filename,index=None)

In [ ]:
generate_submission(preprocess_data,model,"my_submission.csv",threshold=0.5)

In [ ]:
!head my_submission.csv